## 聚类结果直接用于预测

利用聚类结果的转移概率矩阵直接预测

In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from cluster.models import KMeans, MiniBatchKMeans, SpectralClustering, AffinityPropagation
from cluster.dataset import load_time_series
from cluster.analysis import transition_probability_matrix

from bayes_opt import BayesianOptimization

In [3]:
# load data
data = load_time_series(1)
data = data.groupby('datetime')['pwr'].sum()
data = pd.DataFrame(data.values.reshape(-1, 48), index=np.unique(pd.to_datetime(data.index).date.astype(str)))

In [4]:
# 单步预测，通过计算历史标签的转移概率矩阵 m对次日数值进行预测
def predict(x, k=5):
    kmeans_pp = KMeans(
    n_clusters=k,  # 聚类簇数
    max_iter=300,  # 最大迭代次数
    n_init=10,  # 随机初始化运行总次数，反馈最佳聚类结果
    init='k-means++',  # {'kmeans++', 'random', ndarray}
    algorithm='auto',  # {'auto', 'full': EM算法, 'elkan': 应用三角不等式，不支持sparse}
)
    kmeans_pp.fit(x)
    m = transition_probability_matrix(kmeans_pp.labels_)
    x_day = x.sum(axis=1)
    labels_mean = {
        label: np.median(x_day[np.where(kmeans_pp.labels_==label)]) for label in np.unique(kmeans_pp.labels_)
    }
    pred = sum(labels_mean[label] * p for label, p in m.iloc[kmeans_pp.labels_[-1]].items())
    return pred

In [8]:
# 多轮预测封装
x = data.values
def test(k=5, init=100, window=100):
    k = int(k)
    init = int(init)
    window = int(window)
    res = []
    for i in range(init, x.shape[0]):
        res.append(predict(x[i-window:i, :], k))
    y = x.sum(axis=1)[init:]
    pred = np.array(res)
    mape = np.mean(np.abs(pred - y) / y)
    return -mape

def test(k=5, window=100, init=100):
    k = int(k)
    window = int(window)
    res = []
    for i in range(init, x.shape[0]):
        res.append(predict(x[i-window:i, :], k))
    y = x.sum(axis=1)[init:]
    pred = np.array(res)
    mape = np.mean(np.abs(pred - y) / y)
    return -mape

In [16]:
opt = BayesianOptimization(test, {'k': (1, 10),
                                                'init': (50, 100),
                                                'window': (10, 50),
                                                })

In [21]:
opt.maximize(init_points=20, n_iter=30)

Bayesian Optimization
-----------------------------------------------------------------
 Step |   Time |      Value |      init |         k |    window | 
   20 | 00m14s |   -0.09634 |   63.7953 |    8.7818 |   33.6044 | 
   21 | 00m13s |   -0.08969 |   79.5010 |    6.2955 |   19.1591 | 
   22 | 00m13s |   -0.09071 |   56.1525 |    6.0534 |   18.3432 | 
   23 | 00m13s |   -0.08942 |   94.6881 |    6.5297 |   13.7964 | 
   24 | 00m13s |   -0.09028 |   86.9300 |    8.8414 |   10.0297 | 
   25 | 00m12s |   -0.08879 |   78.2377 |    5.4152 |   10.1379 | 
   26 | 00m12s |   -0.08658 |   94.1042 |    5.7506 |   10.0074 | 
   27 | 00m11s |   -0.08776 |   99.8925 |    6.0854 |   10.2235 | 
   28 | 00m11s |   -0.09265 |   50.0241 |    3.7380 |   21.6714 | 
   29 | 00m13s |   -0.09234 |   68.9035 |    7.0643 |   18.6395 | 
   30 | 00m12s |   -0.09130 |   79.4751 |    4.2742 |   29.5772 | 
   31 | 00m12s |   -0.08602 |   91.9264 |    3.8177 |   10.0785 | 
   32 | 00m11s |   -0.09407 |   84.9926 |

In [10]:
# 测试
score = test(k=3, window=10, init=200)
print(f'test dataset mape: {score:.4f}')

test dataset mape: -0.0867
